In [ ]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split

#importing dataset
df_train = pd.read_csv(str( 'sample_products.csv'),sep=',')
df_test = pd.read_csv(str( 'test_products.csv'), sep=',')


1o passo Removal of Stop Words
2o passo Tokenization
3o passo Stemming


In [ ]:
# concatening title and tags
df_copy = df_train.copy()
df_copy["text"] = df_copy["concatenated_tags"] + " " + df_copy["query"]+ " " + df_copy["title"]
df_copy = df_copy[df_copy["concatenated_tags"].notnull()]

In [ ]:
# tokenization
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
df_copy['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_copy['text']] 
print(df_copy['tokenized_text'].head(10))

In [ ]:
# Removal of Stop Words
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
print("stop words")

print(len(stopwords))

#lista de preposições essenciais
prep_essen = ['a', 'ante', 'após', 'até', 'com', 'contra', 'de', 'desde', 'em', 'entre', 'para', 'per',
              'perante', 'por', 'sem', 'sob', 'sobre', 'trás']

#inclusão de combinações de preposições e artigos


prep_essen += ['pelo','ao','pro','do','no','pela','à','pra','da','na',
               'pelos','aos','pros','dos','nos','pelas','às','pras','das','nas']

#remoção de preposições com mesma composição de artigos e substantivos

prep_essen.remove('a')
prep_essen.remove('trás')
prep_essen.remove('pelo')
prep_essen.remove('pelos')


#remoção da lista de stopwords as preposições selecionadas


for word in prep_essen:
    if word in stopwords:
        stopwords.remove(word)

print(len(stopwords))

df_copy['tokens'] = df_copy['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df_copy['tokens'] = df_copy['tokens'].str.lower()
print(df_copy['tokens'].head(10))
print(df_copy['tokens'][4])




In [ ]:
# tokenization
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
df_copy['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_copy['tokens']] 

def my_tokenizer(sentence):
    split_sentence = sentence.split(' ')
    print(split_sentence)
    for index in range(0,len(split_sentence)-1):
        if split_sentence[index] in prep_essen:
            token = split_sentence[index]
            token += ' ' + split_sentence[index+1]
            split_sentence[index] = token
            del split_sentence[index+1]
    print(split_sentence)
    return split_sentence
df_copy['title'] = df_copy['title'].str.lower()

df_copy['t_text'] = df_copy['title'].apply(my_tokenizer)
print(df_copy['t_text'].head(10))

In [ ]:

# Stemming 
import nltk.stem
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()
# Get the stemmed_tokens
df_copy['stemmed_tokens'] = [[stemmer.stem(word) for word in tokens] for tokens in df_copy['tokenized_text']]
df_copy['stemmed_tokens'].head(10)

In [ ]:
# building dictionaries

from gensim import corpora
# Build the dictionary
mydict = corpora.Dictionary(df_copy['stemmed_tokens'])
print("Total unique words:")
print(len(mydict.token2id))
print("\nSample data from dictionary:")
i = 0
# Print top 4 (word, id) tuples
for key in mydict.token2id.keys():
    print("Word: {} - ID: {} ".format(key, mydict.token2id[key]))
    if i == 3:
        break
    i += 1

In [ ]:
#Generating Bow Vectors

import gensim
vocab_len = len(mydict)
print("Example of how the BOW words")
arr = []
for line in df_copy['stemmed_tokens']:
    print("Doc2Bow Line:")
    print(mydict.doc2bow(line))
    for word in line:
        arr.append(mydict.token2id[word])
    print("Actual line:")
    print(line)
    print("(Word, count) Tuples:")
    print([(mydict[id], count) for id, count in mydict.doc2bow(line) ])
    print("Sparse bow vector for the line")
    print(gensim.matutils.corpus2csc([mydict.doc2bow(line)],num_terms=vocab_len).toarray()[:,0])
    break
print("Sorted word id list")
print(sorted(arr))

df_copy = df_copy.fillna(0)

print(df_copy.info())

In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
top_data_df_small = df_copy
def split_train_test(top_data_df_small, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(top_data_df_small[['product_id', 'seller_id','search_page','position', 'creation_date', 'price','weight','express_delivery','minimum_quantity','view_counts','order_counts', 'stemmed_tokens']], 
                                                        top_data_df_small['category'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train set")
    print(Y_train.value_counts())
    print("Value counts for Test set")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

In [ ]:
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1
OUTPUT_FOLDER=''
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_'  +str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(top_data_df_small['stemmed_tokens']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, workers = workers,vector_size = size, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

In [ ]:
import numpy as np
# Load the model from the model file
sg_w2v_model = Word2Vec.load(word2vec_model_file)
# Unique ID of the word
print("Index of the word 'mdf':")
print(sg_w2v_model.wv.key_to_index["mdf"])
# Total number of the words 
print(len(sg_w2v_model.wv))
# Print the size of the word2vec vector for one word
print("Length of the vector generated for a word")
normed_vector = sg_w2v_model.wv.get_vector("mdf", norm=True)
print(len(normed_vector))
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
print(np.mean([len(sg_w2v_model.wv.get_vector(token, norm=True)) for token in top_data_df_small['stemmed_tokens'][0]], axis=0))


In [ ]:
# Store the vectors for train data in following file
OUTPUT_FOLDER =''
word2vec_filename = OUTPUT_FOLDER + 'train_review_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in X_train.iterrows():
        model_vector = (np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in row['stemmed_tokens']], axis=0)).tolist()
        if index == 0:
            ##############ERRO AQUI #############################
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [ ]:
import time
#Import the DecisionTreeeClassifier
from sklearn.tree import DecisionTreeClassifier
# Load from the filename
word2vec_df = pd.read_csv(word2vec_filename)
#Initialize the model
clf_decision_word2vec = DecisionTreeClassifier()

start_time = time.time()
# Fit the model
clf_decision_word2vec.fit(word2vec_df, Y_train['category'])
print("Time taken to fit the model with word2vec vectors: " + str(time.time() - start_time))


In [ ]:
from sklearn.metrics import classification_report
test_features_word2vec = []
for index, row in X_test.iterrows():
    model_vector = np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in row['stemmed_tokens']], axis=0)
    if type(model_vector) is list:
        test_features_word2vec.append(model_vector)
    else:
        test_features_word2vec.append(np.array([0 for i in range(1000)]))
test_predictions_word2vec = clf_decision_word2vec.predict(test_features_word2vec)
print(classification_report(Y_test['category'],test_predictions_word2vec))
